# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Which supply chain stages should we target to reduce food loss and waste and benefit the environment?	Conference presentation	talk-esaussee2019	Ecological Society of America/U.S. Society for Ecological Economics joint meeting	2019-08-14	"Louisville, Kentucky"		
Measuring geodiversity to explain biodiversity: what is the effect of spatial grain and spatial 'consciousness?'	Conference presentation	talk-iale2018	International Association of Landscape Ecology	2018-04-09	"Chicago, Illinois"		
Intraspecific variation reflects drivers of rodent community assembly across the National Ecological Observatory Network	Conference presentation	talk-esa2017	Ecological Society of America	2017-08-08	"Portland, Oregon"		
A globally replicated experiment shows that long-term environmental filters constrain plant response to increased temperature and loss of foundation species	Conference presentation	talk-esa2015	Ecological Society of America	20

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Which supply chain stages should we target to ...,Conference presentation,talk-esaussee2019,Ecological Society of America/U.S. Society for...,2019-08-14,"Louisville, Kentucky",NaN,NaN
1,Measuring geodiversity to explain biodiversity...,Conference presentation,talk-iale2018,International Association of Landscape Ecology,2018-04-09,"Chicago, Illinois",NaN,NaN
2,Intraspecific variation reflects drivers of ro...,Conference presentation,talk-esa2017,Ecological Society of America,2017-08-08,"Portland, Oregon",NaN,NaN
3,A globally replicated experiment shows that lo...,Conference presentation,talk-esa2015,Ecological Society of America,2015-08-11,"Baltimore, Maryland",NaN,NaN
4,Food waste impacts on biodiversity,Invited talk,talk-cec2018,Commission on Environmental Cooperation,2018-10-24,"Arlington, Virginia",NaN,at Food Loss and Waste Measurement Experts Gro...
5,Challenges in the functional trait approach to...,Invited talk,talk-idiv2017,German Centre for Integrative Biodiversity Res...,2017-11-15,"Leipzig, Germany",NaN,NaN
6,Intraspecific variation and community assembly,Invited talk,talk-neon2017,"National Ecological Observatory Network, weekl...",2017-01-05,"Boulder, Colorado",NaN,Co-delivered with John Grady
7,Individual variation in organismal traits: pre...,Invited talk,talk-msu2016,"Michigan State University, Hanover Forest Scie...",2016-09-20,"East Lansing, Michigan",NaN,NaN
8,C3UBE undergraduate biology curriculum reform,Invited talk,talk-nd2015,"University of Notre Dame, Biology education se...",2015-03-01,"Notre Dame, Indiana",NaN,NaN
9,"Roots, leaves, and soils facing global change",Invited talk,talk-rmbl2014,"Rocky Mountain Biological Laboratory, weekly s...",2014-06-01,"Gothic, Colorado",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

In [5]:
# Fix the issues with publication date (excel)
str(talks['date'])

'0     2019-08-14\n1     2018-04-09\n2     2017-08-08\n3     2015-08-11\n4     2018-10-24\n5     2017-11-15\n6     2017-01-05\n7     2016-09-20\n8     2015-03-01\n9     2014-06-01\n10    2014-01-15\nName: date, dtype: object'

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [7]:
!ls ../_talks

2014-01-15-talk-ornl2014.md
2014-06-01-talk-rmbl2014.md
2015-03-01-talk-nd2015.md
2015-08-11-talk-esa2015.md
2016-09-20-talk-msu2016.md
2017-01-05-talk-neon2017.md
2017-08-08-talk-esa2017.md
2017-11-15-talk-idiv2017.md
2018-04-09-talk-iale2018.md
2018-10-24-talk-cec2018.md
2019-08-14-talk-esaussee2019.md


In [8]:
!cat ../_talks/2019-08-14-talk-esaussee2019.md

---
title: "Which supply chain stages should we target to reduce food loss and waste and benefit the environment?"
collection: talks
type: "Conference presentation"
permalink: /talks/2019-08-14-talk-esaussee2019
venue: "Ecological Society of America/U.S. Society for Ecological Economics joint meeting"
date: 2019-08-14
location: "Louisville, Kentucky"
---
